In [127]:
import pandas as pd


In [128]:
DATASET_PATH = '../datasets/downsampled_50_50.csv'
EXPORTED_GENERATIONS_PATH = '../datasets/manual_gpt3_50_50_generations.csv'
PROMPT_NAME = 'match_prompt_manual_fewshot_1_eval2_temp_0_gpt3.5-0301'

In [129]:
# Replace 'your_file.csv' with the actual CSV file path
df = pd.read_csv(EXPORTED_GENERATIONS_PATH)


In [130]:
# remove column "metadata"
df = df.drop(columns=['metadata'])
df = df.drop(columns=['name'])

# add column "eval"

df['eval'] = ""



In [131]:
df

,traceId,model,startTime,endTime,prompt,completion,eval
0,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:25:25.429Z,2023-10-17T10:25:26.263Z,"[[{""content"":""\n These two products are not...","""No match.""",
1,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:25:22.070Z,2023-10-17T10:25:25.422Z,"[[{""content"":""Q: Could you please tell me if t...","""These two products are not the same because t...",
2,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:25:17.126Z,2023-10-17T10:25:17.991Z,"[[{""content"":""\n Yes, these two products ar...","""Match.""",
3,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:25:12.823Z,2023-10-17T10:25:17.121Z,"[[{""content"":""Q: Could you please tell me if t...","""Yes, these two products are the same one. The...",
4,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:25:07.908Z,2023-10-17T10:25:08.549Z,"[[{""content"":""\n Yes, these two products ar...","""Match.""",
...,...,...,...,...,...,...,...
437,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T09:51:37.814Z,2023-10-17T09:51:42.110Z,"[[{""content"":""Q: Could you please tell me if t...","{""completion"":""These products are not the same...",
438,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T09:51:33.415Z,2023-10-17T09:51:34.133Z,"[[{""content"":""\n These two products are not...","{""completion"":""No match.""}",
439,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T09:51:27.877Z,2023-10-17T09:51:33.410Z,"[[{""content"":""Q: Could you please tell me if t...","{""completion"":""These two products are not the ...",
440,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T09:51:23.558Z,2023-10-17T09:51:24.330Z,"[[{""content"":""\n These two products are not...","{""completion"":""No match.""}",


In [132]:
def merge_rows(row1, row2):
    merged_row = {}
    # copy all columns from row2 and add to merged_row
    for column in row2.keys():
        merged_row[column] = row2[column]
    merged_row['eval'] = row1['completion'] 
    return merged_row


In [133]:
merged_data = []
for i in range(0, len(df), 2):
    if i + 1 < len(df):
        merged_row = merge_rows(df.iloc[i], df.iloc[i + 1])
        merged_data.append(merged_row)

merged_df = pd.DataFrame(merged_data)


In [134]:
# sort by column "startTime" in ascending order
# merged_df = merged_df.sort_values(by=['startTime'])

In [135]:
df_dataset = pd.read_csv(DATASET_PATH)

In [136]:
# add column "label" and "predicted_label"
merged_df['predicted_label'] = ""
merged_df['label'] = ""
merged_df['error'] = ""

# copy column "label" from df_dataset to merged_df
for i in range(len(merged_df)):
    label = df_dataset['label'][len(df_dataset)-i-1]
    merged_df['label'][i] = label
    predicted_label = df_dataset[PROMPT_NAME][len(df_dataset)-i-1]
    merged_df['predicted_label'][i] = predicted_label
    if label != predicted_label:
        if label == False:
            merged_df['error'][i] = "FP"
        elif label == True:
            merged_df['error'][i] = "FN"
        else:
            print("ERROR: " + label)
    else:
        merged_df['error'][i] = "NONE"



In [137]:
merged_df

,traceId,model,startTime,endTime,prompt,completion,eval,predicted_label,label,error
0,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:25:22.070Z,2023-10-17T10:25:25.422Z,"[[{""content"":""Q: Could you please tell me if t...","""These two products are not the same because t...","""No match.""",False,True,FN
1,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:25:12.823Z,2023-10-17T10:25:17.121Z,"[[{""content"":""Q: Could you please tell me if t...","""Yes, these two products are the same one. The...","""Match.""",True,True,NONE
2,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:25:04.230Z,2023-10-17T10:25:07.905Z,"[[{""content"":""Q: Could you please tell me if t...","""Yes, these two products are the same. They ar...","""Match.""",True,True,NONE
3,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:24:56.337Z,2023-10-17T10:24:59.610Z,"[[{""content"":""Q: Could you please tell me if t...","""These two products are likely the same becaus...","""Match.""",True,True,NONE
4,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:24:48.374Z,2023-10-17T10:24:51.714Z,"[[{""content"":""Q: Could you please tell me if t...","""Yes, these two products are the same one. The...","""Match.""",True,True,NONE
...,...,...,...,...,...,...,...,...,...,...
216,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T09:51:58.234Z,2023-10-17T09:52:02.222Z,"[[{""content"":""Q: Could you please tell me if t...","{""completion"":""These two products are not the ...","{""completion"":""No match.""}",False,False,NONE
217,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T09:51:46.656Z,2023-10-17T09:51:54.143Z,"[[{""content"":""Q: Could you please tell me if t...","{""completion"":""These two products are not the ...","{""completion"":""No match.""}",False,False,NONE
218,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T09:51:37.814Z,2023-10-17T09:51:42.110Z,"[[{""content"":""Q: Could you please tell me if t...","{""completion"":""These products are not the same...","{""completion"":""No match.""}",False,False,NONE
219,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T09:51:27.877Z,2023-10-17T09:51:33.410Z,"[[{""content"":""Q: Could you please tell me if t...","{""completion"":""These two products are not the ...","{""completion"":""No match.""}",False,False,NONE


In [138]:
# Replace 'merged_file.csv' with the desired output file name
merged_df.to_csv(EXPORTED_GENERATIONS_PATH.replace(".csv","") + '_merged.csv', index=True)
